# Roku Experiments

## Resources
  - https://github.com/flirc/sdk/blob/master/cli/src/cmds/ir_transmit.c
  - https://github.com/hulu/roku-dev-cli/blob/master/roku_dev_cli/roku_dev_cli.py
  - https://gist.github.com/peterjc/7422624
  - https://www.exploitee.rs/index.php/Exploiting_Roku

In [14]:

OUI = [
  "A8B57C", "BCD7D4", "20EFBD", "CC6DA0",
  "DC3A5E", "B83E59", "AC3A7A"
  ]



In [13]:
import requests

ROKU_IP = "192.168.1.210"

requests.post(f"http://{ROKU_IP}:8060/keypress/home")


<Response [200]>